In [1]:
from datasets import load_dataset

def load_verigui():
    ds_dict = load_dataset("2077AIDataFoundation/VeriGUI")
    # Use the only available split: "test"
    ds = ds_dict["test"]
    print(f"Loaded VeriGUI with {len(ds)} tasks (split='test').")
    return ds

ds = load_verigui()
ds[0]  # preview first example


c:\Users\hala\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\hala\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:121: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\hala\.cache\huggingface\hub\datasets--2077AIDataFoundation--VeriGUI. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as

Loaded VeriGUI with 302 tasks (split='test').


{'folder': 'V3_33',
 'instruct': "Count the cumulative number of visitors to Asia's largest theme parks from 2015 to 2024 (if data for 2024 is unavailable, collect data for available years only), identify the park with the highest visitor count, and list the park's name, city, total visitor count, year with the highest annual visitor count, name and opening date of the signature new attraction introduced that year, price increase for single-day tickets for ages 12 and above (excluding price changes due to special circumstances, with specific ticket price changes based on local price adjustments), operating company, and largest shareholder.",
 'result': ['visitor attendance data of major theme parks in Asia from 2015 to 2024 ,  Tokyo Disneyland 2015: 16,600,000 2016: 16,540,000 2017: 16,600,000 2018: 17,907,000 2019: 17,910,000 2020: 4,160,000 2021: 6,300,000 2022: 12,000,000 2023: 15,100,000  Tokyo DisneySea 2015: 12,100,000 2016: 13,460,000 2017: 13,500,000 2018: 14,651,000 2019: 14,6

In [2]:
# # Map agent names to integer IDs
# AGENT_IDS = {"nav_agent": 0, "pers_agent": 1, "llm_agent": 2}

# def infer_agent_for_subtask(sub_instruction):
#     """Heuristic agent assignment based on subtask instruction"""
#     inst = sub_instruction.lower()
#     if any(k in inst for k in ['search', 'find', 'look up', 'navigate', 'click', 'scroll']):
#         return 'nav_agent'
#     if any(k in inst for k in ['filter', 'analyze', 'compare', 'extract', 'identify', 'plan']):
#         return 'llm_agent'
#     if any(k in inst for k in ['suggest', 'recommend', 'adapt', 'personalize']):
#         return 'pers_agent'
#     # default alternating
#     return 'llm_agent'

# def create_numeric_state(task_instruction, completed_subtasks, current_subtask, step_idx, total_steps):
#     """Create a numeric state vector"""
#     state = []

#     # Progress
#     progress = step_idx / total_steps if total_steps > 0 else 0.0
#     state.append(progress)

#     # Task complexity
#     complexity = min(len(completed_subtasks)/10.0, 1.0)
#     state.append(complexity)

#     # Instruction length
#     instr_len = min(len(task_instruction)/500.0, 1.0)
#     state.append(instr_len)

#     # Current subtask length
#     sub_len = min(len(current_subtask)/200.0, 1.0)
#     state.append(sub_len)

#     # Completed subtasks count (normalized)
#     state.append(min(len(completed_subtasks)/10.0,1.0))

#     # Available agents (all initially available)
#     state.extend([1.0, 1.0, 1.0])  # nav, pers, llm

#     # Step number normalized
#     state.append(min(step_idx/20.0, 1.0))

#     # Pad to fixed length
#     while len(state) < 16:
#         state.append(0.0)

#     return state[:16]

# def calculate_step_reward(step_idx, total_steps, agent):
#     """Shaped reward for RL"""
#     reward = 0.1  # small base reward

#     if step_idx == 0 and agent == 'llm_agent':
#         reward += 0.2
#     if step_idx == total_steps - 1 and agent == 'pers_agent':
#         reward += 0.2
#     if step_idx > 1:
#         reward -= 0.05  # penalize repeated steps

#     return max(reward, 0.01)

# def calculate_returns_to_go(rewards, gamma=0.99):
#     """Compute returns-to-go"""
#     returns = []
#     g = 0.0
#     for r in reversed(rewards):
#         g = r + gamma * g
#         returns.insert(0, g)
#     return returns


In [3]:
# import json

# def prepare_rl_transitions(ds):
#     transitions = []

#     for item in ds:
#         task_instruction = item["instruct"]
#         subtasks = item["sub_tasks"]
#         total_steps = len(subtasks)

#         for i, st in enumerate(subtasks):
#             current_instruction = st["instruct"]
#             agent_name = infer_agent_for_subtask(current_instruction)
#             action_id = AGENT_IDS[agent_name]

#             numeric_state = create_numeric_state(
#                 task_instruction, [s["result"] for s in subtasks[:i]], current_instruction, i, total_steps
#             )

#             reward = calculate_step_reward(i, total_steps, agent_name)
#             done = (i == total_steps - 1)

#             transition = {
#                 "state": numeric_state,
#                 "task_instruction": task_instruction,
#                 "current_subtask_instruction": current_instruction,
#                 "completed_subtasks": [s["result"] for s in subtasks[:i]],
#                 "action": action_id,
#                 "expected_output": st["result"],
#                 "reward": reward,
#                 "done": done
#             }

#             transitions.append(transition)

#     print("Prepared", len(transitions), "RL transitions with numeric states and shaped rewards.")
#     return transitions

# rl_data = prepare_rl_transitions(ds)
# rl_data[:2]  # preview first 2 transitions


In [4]:
# import json
# import random

# # ============================================================
# # CONFIG
# # ============================================================

# AGENT_IDS = {
#     "nav_agent": 0,
#     "pers_agent": 1,
#     "llm_agent": 2
# }

# NUM_AGENTS = len(AGENT_IDS)
# STATE_DIM = 16


# # ============================================================
# # STATE CONSTRUCTION (NO ORACLE SIGNALS)
# # ============================================================

# def create_numeric_state(
#     task_instruction,
#     completed_subtasks,
#     current_subtask_instruction,
#     step_idx,
#     total_steps
# ):
#     """
#     Create an RL observation vector WITHOUT leaking answers.
#     """

#     state = []

#     # Progress through task (allowed)
#     state.append(step_idx / max(total_steps, 1))

#     # Number of completed subtasks (normalized)
#     state.append(len(completed_subtasks) / max(total_steps, 1))

#     # Task instruction length (proxy for difficulty)
#     state.append(min(len(task_instruction) / 500.0, 1.0))

#     # Current subtask length
#     state.append(min(len(current_subtask_instruction) / 200.0, 1.0))

#     # Total number of subtasks (normalized)
#     state.append(min(total_steps / 10.0, 1.0))

#     # Agent availability (all available by default)
#     state.extend([1.0] * NUM_AGENTS)

#     # Remaining steps ratio
#     remaining = (total_steps - step_idx - 1) / max(total_steps, 1)
#     state.append(max(remaining, 0.0))

#     # Padding
#     while len(state) < STATE_DIM:
#         state.append(0.0)

#     return state[:STATE_DIM]


# # ============================================================
# # DATASET PREPARATION (NO ACTIONS, NO REWARDS)
# # ============================================================

# def prepare_rl_dataset(raw_dataset):
#     """
#     Prepare dataset for RL environment consumption.

#     Output format:
#     Each entry represents a decision point.
#     """

#     samples = []

#     for item in raw_dataset:
#         task_instruction = item["instruct"]
#         subtasks = item["sub_tasks"]
#         total_steps = len(subtasks)

#         completed_results = []

#         for step_idx, sub in enumerate(subtasks):

#             obs = create_numeric_state(
#                 task_instruction=task_instruction,
#                 completed_subtasks=completed_results,
#                 current_subtask_instruction=sub["instruct"],
#                 step_idx=step_idx,
#                 total_steps=total_steps
#             )

#             sample = {
#                 # Observation
#                 "state": obs,

#                 # Text context (for env / embedding model)
#                 "task_instruction": task_instruction,
#                 "current_subtask_instruction": sub["instruct"],
#                 "completed_subtasks": completed_results.copy(),

#                 # Environment control signals
#                 "step_index": step_idx,
#                 "total_steps": total_steps,
#                 "done": (step_idx == total_steps - 1),

#                 # Ground truth execution result (NOT used for reward here)
#                 "expected_output": sub["result"],

#                 # Success flag placeholder (computed by environment)
#                 "success": None
#             }

#             samples.append(sample)
#             completed_results.append(sub["result"])

#     print(f"Prepared {len(samples)} RL decision points.")
#     return samples


# # ============================================================
# # OPTIONAL: TRAIN / VAL SPLIT
# # ============================================================

# def split_dataset(samples, val_ratio=0.1, seed=42):
#     random.seed(seed)
#     random.shuffle(samples)

#     split_idx = int(len(samples) * (1 - val_ratio))
#     return samples[:split_idx], samples[split_idx:]


# # ============================================================
# # USAGE
# # ============================================================

# # rl_samples = prepare_rl_dataset(ds)
# # train_data, val_data = split_dataset(rl_samples)

# # ============================================================
# # USAGE & SAVE TO FILE
# # ============================================================

# # If your dataset is already loaded in memory as `ds`, no need to read a file
# # ds = ...  # your dataset in memory

# # Prepare RL dataset
# rl_samples = prepare_rl_dataset(ds)

# # Optional: train/validation split
# train_data, val_data = split_dataset(rl_samples, val_ratio=0.1)

# # Save full RL dataset
# OUTPUT_FILE_ALL = "verigui_coordinator_rlnew.json"
# with open(OUTPUT_FILE_ALL, "w", encoding="utf-8") as f:
#     json.dump(rl_samples, f, indent=2, ensure_ascii=False)
# print(f"✅ Saved full RL dataset to: {OUTPUT_FILE_ALL}")

# # Save train/validation splits
# OUTPUT_FILE_TRAIN = "verigui_coordinator_rl_train.json"
# OUTPUT_FILE_VAL = "verigui_coordinator_rl_val.json"

# with open(OUTPUT_FILE_TRAIN, "w", encoding="utf-8") as f:
#     json.dump(train_data, f, indent=2, ensure_ascii=False)
# with open(OUTPUT_FILE_VAL, "w", encoding="utf-8") as f:
#     json.dump(val_data, f, indent=2, ensure_ascii=False)

# print(f"✅ Saved train split to: {OUTPUT_FILE_TRAIN}")
# print(f"✅ Saved validation split to: {OUTPUT_FILE_VAL}")


In [5]:
# import json

# OUTPUT_FILE = "verigui_coordinator_rlnew.json"

# # Save only if we have data
# if rl_samples and len(rl_samples) > 0:
#     with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
#         json.dump(rl_samples, f, indent=2, ensure_ascii=False)
#     print(f"✅ Saved RL-ready dataset to: {OUTPUT_FILE}")
# else:
#     print("⚠️ No RL samples to save! Check your dataset preprocessing.")


In [6]:
# OUTPUT_FILE = "verigui_coordinator_rlnew.json"

# with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
#     json.dump(rl_data, f, indent=2, ensure_ascii=False)

# print("✅ Saved RL-ready dataset to:", OUTPUT_FILE)


claude modifications: 15/1/2025  12:47 am

In [10]:
import json

# ============================================================
# CONFIG
# ============================================================

GAMMA = 0.99
STATE_DIM = 12
ACT_DIM = len(ACTION_SPACE)  # Automatically matches coordinator

AGENT_IDS = {
    "nav_agent": 0,
    "pers_agent": 1,
    "llm_agent": 2
}
NUM_AGENTS = len(AGENT_IDS)

# ============================================================
# STATE CONSTRUCTION (12-DIM)
# ============================================================

def create_numeric_state(
    task_instruction,
    completed_subtasks,
    current_subtask_instruction,
    step_idx,
    total_steps
):
    state = []

    # Temporal progress
    state.append(step_idx / max(total_steps, 1))
    state.append((total_steps - step_idx - 1) / max(total_steps, 1))

    # Completion ratio
    state.append(len(completed_subtasks) / max(total_steps, 1))

    # Instruction complexity
    state.append(min(len(task_instruction) / 500.0, 1.0))
    state.append(min(len(current_subtask_instruction) / 200.0, 1.0))

    # Agent availability
    state.extend([1.0] * NUM_AGENTS)

    # Padding
    while len(state) < STATE_DIM:
        state.append(0.0)

    return state[:STATE_DIM]

# ============================================================
# REWARD FUNCTION
# ============================================================

def compute_reward(prev_progress, new_progress, done, success):
    reward = 0.0

    # Progress shaping
    reward += (new_progress - prev_progress)

    # Forward movement bonus
    if new_progress > prev_progress:
        reward += 0.1

    # Terminal reward
    if done:
        reward += 1.0 if success else -1.0

    return reward

# ============================================================
# DATASET PREPARATION (COORDINATOR-ALIGNED)
# ============================================================

def prepare_rl_dataset(raw_dataset):
    transitions = []
    episode_id = 0

    for task_id, item in enumerate(raw_dataset):
        task_instruction = item["instruct"]
        subtasks = item.get("sub_tasks", [])
        total_steps = len(subtasks)

        completed_results = []
        prev_state = None
        prev_progress = 0.0
        episode_buffer = []

        for step_idx, sub in enumerate(subtasks):
            state = create_numeric_state(
                task_instruction,
                completed_results,
                sub.get("instruct", ""),
                step_idx,
                total_steps
            )

            # -----------------------------
            # Semantic action mapping
            # -----------------------------
            action = infer_action_from_subtask(sub.get("instruct", ""))

            progress = step_idx / max(total_steps, 1)
            done = (step_idx == total_steps - 1)
            success = done  # Offline assumption: last step succeeds

            reward = compute_reward(prev_progress, progress, done, success)

            if prev_state is not None:
                episode_buffer.append({
                    "task_id": task_id,
                    "episode_id": episode_id,
                    "step_id": step_idx,
                    "state": prev_state,
                    "action": action,
                    "reward": reward,
                    "next_state": state,
                    "done": done,
                    "return_to_go": None,
                    "action_name": ACTION_SPACE[action],
                    "termination_type": "success" if done and success else None
                })

            prev_state = state
            prev_progress = progress
            completed_results.append(sub.get("result", ""))

        # ----------------------------------------------------
        # RETURN-TO-GO COMPUTATION
        # ----------------------------------------------------
        G = 0.0
        for t in reversed(episode_buffer):
            G = t["reward"] + GAMMA * G
            t["return_to_go"] = G

        transitions.extend(episode_buffer)
        episode_id += 1

    print(f"Prepared {len(transitions)} RL transitions.")
    return transitions

# ============================================================
# TRAIN / VAL SPLIT
# ============================================================

def split_dataset(samples, val_ratio=0.1, seed=42):
    import random
    random.seed(seed)
    random.shuffle(samples)
    split_idx = int(len(samples) * (1 - val_ratio))
    return samples[:split_idx], samples[split_idx:]

# ============================================================
# USAGE
# ============================================================

# ds must be loaded beforehand:
# with open("raw_dataset.json") as f:
#     ds = json.load(f)

rl_transitions = prepare_rl_dataset(ds)
train_data, val_data = split_dataset(rl_transitions)

with open("verigui_coordinator_rl_train.json", "w", encoding="utf-8") as f:
    json.dump(train_data, f, indent=2, ensure_ascii=False)

with open("verigui_coordinator_rl_val.json", "w", encoding="utf-8") as f:
    json.dump(val_data, f, indent=2, ensure_ascii=False)

print("✅ RL dataset fully aligned with coordinator action semantics.")


Prepared 994 RL transitions.
✅ RL dataset fully aligned with coordinator action semantics.
